In [13]:
import os

import pandas as pd
from sqlalchemy import create_engine
from dotenv import load_dotenv

load_dotenv()


def batch_load_sql(query: str) -> pd.DataFrame:
    CHUNKSIZE = 200000
    engine = create_engine(os.getenv('DATABASE'))
    conn = engine.connect().execution_options(stream_results=True)
    chunks = []
    for chunk_dataframe in pd.read_sql(query, conn, chunksize=CHUNKSIZE):
        chunks.append(chunk_dataframe)
    conn.close()
    return pd.concat(chunks, ignore_index=True)

def load_features(select) -> pd.DataFrame:
    return batch_load_sql(select)


In [14]:
query = """
SELECT *
FROM post_text_df
"""
posts_df = load_features(query)

posts_df.head()

,post_id,text,topic
0,1,UK economy facing major risks\n\nThe UK manufa...,business
1,2,Aids and climate top Davos agenda\n\nClimate c...,business
2,3,Asian quake hits European shares\n\nShares in ...,business
3,4,India power shares jump on debut\n\nShares in ...,business
4,5,Lacroix label bought by US firm\n\nLuxury good...,business


In [15]:
# Find total likes, likes rate for every post

In [16]:
query = """
SELECT COUNT(*), post_id
FROM feed_data
WHERE action = 'like'
GROUP BY post_id;
"""
posts_likes = load_features(query)
posts_likes = posts_likes.set_index('post_id')

In [17]:
posts_df['total_likes'] = posts_df['post_id'].map(posts_likes['count'])

In [18]:
query = """
SELECT COUNT(*), post_id
FROM feed_data
GROUP BY post_id;
"""
posts_actions = load_features(query)
posts_actions = posts_actions.set_index('post_id')

In [19]:
posts_df['like_rate'] = posts_df['post_id'].map(posts_likes['count']/posts_actions['count'])

In [25]:
posts_df.isnull().sum()

post_id        0
text           0
topic          0
total_likes    0
like_rate      0
dtype: int64

In [21]:
posts_df['like_rate'] = posts_df['like_rate'].fillna(posts_df['like_rate'].mean())
posts_df['total_likes'] = posts_df['total_likes'].fillna(posts_df['total_likes'].mean())

In [22]:
# Add feature - average age for each post

In [ ]:
query = """
SELECT AVG(u.age), f.post_id
FROM feed_data as f
JOIN user_data as u
ON f.user_id = u.user_id
JOIN post_text_df as p
ON f.post_id = p.post_id
WHERE action = 'like'
GROUP BY p.post_id
"""
posts_age_likes = load_features(query)
posts_age_likes = posts_age_likes.set_index('post_id')